# Finding out the output of 'match' function

In [1]:
from data import *
from utils.augmentations import SSDAugmentation
import torch
from torch.autograd import Variable
from box_utils import match, point_form, jaccard, match, intersect, encode
from box_utils import log_sum_exp
from SSD_generate_anchors import generate_ssd_priors

import collections
from pathlib import Path
from torch.utils.data import DataLoader
import numpy as np

In [2]:
root = Path.home()/'Documents'/'DATASETS'/'VOCdevkit'
cfg = voc

In [3]:
dataset = VOCDetection(root = root,
                      transform=SSDAugmentation(cfg['min_dim'], MEANS))

In [4]:
batch_size = 7

data_loader = DataLoader(dataset, batch_size,
                          num_workers=2,
                          shuffle=True,
                          collate_fn=detection_collate,
                          pin_memory=True)
batch_iterator = iter(data_loader)

In [5]:
SSDBoxSizes = collections.namedtuple('SSDBoxSizes', ['min', 'max'])

Spec = collections.namedtuple('Spec', ['feature_map_size', 'shrinkage', 'box_sizes', 
                                       'aspect_ratios'])

specs = [
    Spec(38, 8, SSDBoxSizes(30, 60), [2]),
    Spec(19, 16, SSDBoxSizes(60, 111), [2, 3]),
    Spec(10, 32, SSDBoxSizes(111, 162), [2, 3]),
    Spec(5, 64, SSDBoxSizes(162, 213), [2, 3]),
    Spec(3, 100, SSDBoxSizes(213, 264), [2]),
    Spec(1, 300, SSDBoxSizes(264, 315), [2])
]

priors = generate_ssd_priors(specs, clip=True)

In [6]:
priors = torch.Tensor(priors)
# priors = point_form(priors)

In [7]:
priors

tensor([[0.0133, 0.0133, 0.1000, 0.1000],
        [0.0133, 0.0133, 0.1414, 0.1414],
        [0.0133, 0.0133, 0.1414, 0.0707],
        ...,
        [0.5000, 0.5000, 0.9612, 0.9612],
        [0.5000, 0.5000, 1.0000, 0.6223],
        [0.5000, 0.5000, 0.6223, 1.0000]])

In [8]:
for idx, (image, target) in enumerate(batch_iterator):
#     print(image.size(0))
    bs    = image.size(0)
    loc_t = torch.Tensor(batch_size, 8732, 4)
    conf_t = torch.LongTensor(batch_size, 8732)
    
    for i in range(bs):
        truths = target[i][:,:-1].data
        labels = target[i][:,-1].data
        defaults = priors.data

        match(0.6, truths, defaults, [0.1,0.2], labels,
                      loc_t, conf_t, idx)
    
    if idx == 0:
        break

1 8732
3 8732
1 8732
1 8732
1 8732
1 8732
1 8732


In [9]:
labels

tensor([12.])

In [10]:
conf_t.size()

torch.Size([7, 8732])

In [11]:
loc_t.size()

torch.Size([7, 8732, 4])

In [12]:
conf_t

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [13]:
pos = conf_t > 0
print(pos.size())
num_pos = pos.sum(dim=1, keepdim=True)
print(num_pos.size())

torch.Size([7, 8732])
torch.Size([7, 1])


In [14]:
num_pos

tensor([[1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]])

In [15]:
num_pos.size()

torch.Size([7, 1])

In [16]:
random_data = np.random.randn(batch_size, 8732, 4)
loc_data    = torch.Tensor(random_data)
print(loc_data.size())

pos_idx = pos.unsqueeze(pos.dim()).expand_as(loc_data)
print(pos_idx.size())

loc_p = loc_data[pos_idx].view(-1, 4)
loc_t = loc_t[pos_idx].view(-1, 4)

print(loc_p.size())
print(loc_t.size())

torch.Size([7, 8732, 4])
torch.Size([7, 8732, 4])
torch.Size([1, 4])
torch.Size([1, 4])


In [17]:
loc_p


tensor([[ 0.1962, -0.4101,  0.3954,  1.5375]])

In [18]:
loc_t

tensor([[ 1.2862, -1.5149,  1.2765, -0.4414]])

In [19]:
conf_data = np.random.randint(low= 0, high= 20, size= (batch_size, 8732, 21))
conf_data = torch.Tensor(conf_data)

In [20]:
batch_conf = conf_data.view(-1, 21)
loss_c = log_sum_exp(batch_conf) - torch.gather(batch_conf,1, conf_t.view(-1, 1))

In [96]:
print(batch_conf.size())
print(conf_t.view(-1,1).size())

torch.Size([61124, 21])
torch.Size([61124, 1])


In [45]:
# v = np.random.randint(0, 20, size=(10, 21))
# print(v.shape)
# print(v)

(10, 21)
[[ 7 17  2 18  2  1  3  2 15 16  5 12  6 13 13 16  9  3 11  8  2]
 [14  3 17 15  1  8  2  7 18  0 18  3  0 11  4 12 16  8 18 11 15]
 [13  4  4 17 13 14  6  0  7  7  7 13  0 18  8  4  9  3  0 14 14]
 [ 4  7 19  7  7 15  0 13  5 19  6  0  5 15  5 12  3  1 18  8  6]
 [19 18  5  3 11 10  2 12 14  9 16 19  3  1 14 19 19 13 16  8 13]
 [ 7  0  6  2  5  1  8  4 13  2 17 11 17 10  8 17  0 12 17 14 16]
 [18 17  3  6  0  2 14 19 18 13 12 12  1  3 13 16 12 10  8  0 16]
 [17 12 12  0  3 18 14 10  9 18  8 12 14 12  7  7  4 19  2  4 12]
 [15 15 18 14  1  6  3  8 14 10  4 16  8  7  3 12 18 11 11 12  3]
 [17  3  9 10  7 12  9  3 18 18  8  4  3 12  2  5 10  3  5 18  3]]


In [46]:
# v_ = torch.Tensor(v)
# print(v_)

tensor([[ 7., 17.,  2., 18.,  2.,  1.,  3.,  2., 15., 16.,  5., 12.,  6., 13.,
         13., 16.,  9.,  3., 11.,  8.,  2.],
        [14.,  3., 17., 15.,  1.,  8.,  2.,  7., 18.,  0., 18.,  3.,  0., 11.,
          4., 12., 16.,  8., 18., 11., 15.],
        [13.,  4.,  4., 17., 13., 14.,  6.,  0.,  7.,  7.,  7., 13.,  0., 18.,
          8.,  4.,  9.,  3.,  0., 14., 14.],
        [ 4.,  7., 19.,  7.,  7., 15.,  0., 13.,  5., 19.,  6.,  0.,  5., 15.,
          5., 12.,  3.,  1., 18.,  8.,  6.],
        [19., 18.,  5.,  3., 11., 10.,  2., 12., 14.,  9., 16., 19.,  3.,  1.,
         14., 19., 19., 13., 16.,  8., 13.],
        [ 7.,  0.,  6.,  2.,  5.,  1.,  8.,  4., 13.,  2., 17., 11., 17., 10.,
          8., 17.,  0., 12., 17., 14., 16.],
        [18., 17.,  3.,  6.,  0.,  2., 14., 19., 18., 13., 12., 12.,  1.,  3.,
         13., 16., 12., 10.,  8.,  0., 16.],
        [17., 12., 12.,  0.,  3., 18., 14., 10.,  9., 18.,  8., 12., 14., 12.,
          7.,  7.,  4., 19.,  2.,  4., 12.],
        

In [47]:
log_sum_exp(v_)

tensor([[18.5338],
        [19.2880],
        [18.3669],
        [19.8788],
        [20.5013],
        [18.4921],
        [19.6857],
        [19.6362],
        [18.8236],
        [19.2160]])

In [48]:
def np_log_sum_exp(x):
    x_max = np.max(x)
    return np.log(np.sum(np.exp(x - x_max), axis=1, keepdims=True)) + x_max

In [49]:
np_log_sum_exp(v)

array([[18.53379888],
       [19.28799666],
       [18.36691006],
       [19.87877325],
       [20.50125585],
       [18.49210127],
       [19.68574085],
       [19.63622123],
       [18.82363708],
       [19.21604676]])

In [93]:
m = torch.randn(4,2)
# print(m)
ids = torch.Tensor([1,1,0,0]).long()
print(m.gather(1, ids.view(-1,1)))
aa = m.gather(1, ids.view(-1,1)).numpy()

tensor([[-2.0881],
        [ 0.5326],
        [ 0.1850],
        [ 0.4622]])


In [89]:
def gather(self, dim, index):
    """
    Gathers values along an axis specified by ``dim``.

    For a 3-D tensor the output is specified by:
        out[i][j][k] = input[index[i][j][k]][j][k]  # if dim == 0
        out[i][j][k] = input[i][index[i][j][k]][k]  # if dim == 1
        out[i][j][k] = input[i][j][index[i][j][k]]  # if dim == 2

    Parameters
    ----------
    dim:
        The axis along which to index
    index:
        A tensor of indices of elements to gather

    Returns
    -------
    Output Tensor
    """
    idx_xsection_shape = index.shape[:dim] + \
        index.shape[dim + 1:]
    self_xsection_shape = self.shape[:dim] + self.shape[dim + 1:]
    if idx_xsection_shape != self_xsection_shape:
        raise ValueError("Except for dimension " + str(dim) +
                         ", all dimensions of index and self should be the same size")
    if index.dtype != np.dtype('int_'):
        raise TypeError("The values of index must be integers")
    data_swaped = np.swapaxes(self, 0, dim)
    index_swaped = np.swapaxes(index, 0, dim)
    gathered = np.choose(index_swaped, data_swaped)
    return np.swapaxes(gathered, 0, dim)

In [94]:
m_ = m.numpy()
ind = np.array([1,1,0,0]).reshape(-1,1)
bb = gather(m_, 1, ind)

In [95]:
aa == bb

array([[ True],
       [ True],
       [ True],
       [ True]])

In [54]:
np.take(x, index, axis=1)

array([[[3, 2],
        [1, 2]],

       [[6, 5],
        [4, 5]]])

In [21]:
loss_c

tensor([[ 6.8046],
        [16.4863],
        [ 3.0496],
        ...,
        [16.9326],
        [ 8.0821],
        [17.1034]])

In [22]:
loss_c = loss_c.view(batch_size, -1)

In [23]:
loss_c[pos] = 0

In [99]:
_, loss_idx = loss_c.sort(1, descending=True)
print(loss_idx)
print(loss_idx.size())

tensor([[2784, 8110, 2573,  ..., 8298, 3937, 8639],
        [3780, 2037, 4939,  ..., 8472, 2098, 5439],
        [3066, 7372,  660,  ..., 7581, 7409, 2293],
        ...,
        [7516, 6649, 6664,  ..., 4410, 8527, 7302],
        [8424,  511,  512,  ..., 2082, 1354, 5835],
        [3084,  483, 4354,  ..., 4455, 3103,  963]])
torch.Size([7, 8732])


In [115]:
loss_idx_ = np.argsort(loss_c, axis=-1)
loss_idx_ = np.flip(loss_idx_,axis=-1)
print(loss_idx_)
print(loss_idx_.shape)

[[2784 8110 2573 ... 8298 3937 8639]
 [3780 2037 4939 ... 8472 2098 5439]
 [3066 7372  660 ... 7581 7409 2293]
 ...
 [7516 6649 6664 ... 4410 8527 7302]
 [8424  511  512 ... 2082 1354 5835]
 [3084  483 4354 ... 4455 3103  963]]
(7, 8732)


In [116]:
_, idx_rank = loss_idx.sort(1)
print(idx_rank)
print(idx_rank.size())

tensor([[5617, 1357, 7297,  ..., 1305,  626, 1905],
        [5424, 1982, 3186,  ..., 3919,  674, 5072],
        [5064,  167, 4735,  ..., 2778, 2540, 5151],
        ...,
        [2587,  359, 2002,  ..., 4271, 7531, 5650],
        [2831, 5703, 8658,  ..., 5758, 2721, 6096],
        [6803, 4115, 6779,  ..., 1148, 5028, 1092]])
torch.Size([7, 8732])


In [118]:
idx_rank_ = np.argsort(loss_idx_, axis=1)

print(idx_rank_)
print(idx_rank_.shape)

[[5617 1357 7297 ... 1305  626 1905]
 [5424 1982 3186 ... 3919  674 5072]
 [5064  167 4735 ... 2778 2540 5151]
 ...
 [2587  359 2002 ... 4271 7531 5650]
 [2831 5703 8658 ... 5758 2721 6096]
 [6803 4115 6779 ... 1148 5028 1092]]
(7, 8732)


In [114]:
dd = loss_idx.numpy() == loss_idx_
print(sum(sum(dd)))

61104


In [25]:
_, idx_rank = loss_idx.sort(1)
print(idx_rank.size())

torch.Size([7, 8732])


In [26]:
num_pos = pos.sum(1 , keepdim = True)

In [120]:
num_neg = torch.clamp(3 * num_pos, max=pos.size(1) - 1)
print(num_neg)
print(num_neg.size())

tensor([[3],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]])
torch.Size([7, 1])


In [121]:
num_neg_ = np.clip(3 * num_pos.numpy(), a_min=None, a_max= 8731)

In [122]:
print(num_neg)

tensor([[3],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]])


In [28]:
neg = idx_rank < num_neg.expand_as(idx_rank)
print(neg)

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.uint8)


In [29]:
pos_idx = pos.unsqueeze(2).expand_as(conf_data)
neg_idx = neg.unsqueeze(2).expand_as(conf_data)
print(pos_idx.size())
print(neg_idx.size())

torch.Size([7, 8732, 21])
torch.Size([7, 8732, 21])


In [128]:
torch.gt(torch.tensor(21),5)

tensor(1, dtype=torch.uint8)

In [30]:
conf_p = conf_data[(pos_idx+neg_idx).gt(0)].view(-1, 21)
print(conf_p.size())

torch.Size([4, 21])


In [31]:
targets_weighted = conf_t[(pos+neg).gt(0)]

In [32]:
N = num_pos.data.sum()

In [33]:
loss_l /= N
loss_c /= N

NameError: name 'loss_l' is not defined